# Tools

## HAProxy

<img src="./helpers/haproxy-icon.jpeg" alt="drawing" width="600"/>

### Description

A feature rich and very performant (regular or reverse) proxy that is mainly used for High Availability and Load Balancing.

On cloud environment this will be typically replaced by a native proxy.

### Responsibility In Our Environment

- Serve as a single point of interaction for the users of the DB and encapsulate the multi node architecture behind it.
- Load balance between read replicas
- Differentiate between read and write traffic and route the requests to the relevant server considering the current state of the cluster

# Demo

## The Problem

Try write some rows on primary then switchover and write more to the same server.

This of course will result in an error because the prior primary has become the new standby with read-only transactions.

## Architecture

<img src="./helpers/Replication - Load Balance.png" alt="drawing" width="700"/>

## Prerequisites

- 3 Linux machines with PG and Patroni installed
- 3 Linux machines with etcd nodes
- 1 Linux machine (for haproxy, on docker-compose change command to sleep infinity)
- 1 Linux machine with client tools to query postgres

## Setup HAProxy

### Configure

Create a a new `cfg` config file or edit the existing default `/etc/haproxy/haproxy.cfg`

#### HAProxy Global And Defaults

```cfg
global
    maxconn 300
    log     127.0.0.1 local2

defaults
    log global
    mode tcp
    retries 2
    timeout client 30m
    timeout connect 4s
    timeout server 30m
    timeout check 5s

listen stats
    mode http
    bind *:7000
    stats enable
    stats uri /
```

#### Primary

```cfg
listen primary
    bind *:5000
    option httpchk
    http-check expect status 200
    default-server inter 3s fall 3 rise 2 on-marked-down shutdown-sessions
    server pg-1 pg-1:5432 maxconn 100 check port 8008
    server pg-2 pg-2:5432 maxconn 100 check port 8008
    server pg-3 pg-3:5432 maxconn 100 check port 8008
```

#### Standby

```cfg
listen standby
    bind *:5001
    option httpchk OPTIONS/replica
    http-check expect status 200
    default-server inter 3s fall 3 rise 2 on-marked-down shutdown-sessions
    server pg-1 pg-1:5432 maxconn 100 check port 8008
    server pg-2 pg-2:5432 maxconn 100 check port 8008
    server pg-3 pg-3:5432 maxconn 100 check port 8008
```

### Run HAProxy

In [ ]:
haproxy -f /etc/haproxy/haproxy.cfg

## Checkout Our Load Balance

### Play

- Insert rows into 5000 port and watch it created on 5001 port
- Switchover between pg nodes
- Use the write port again and see logs about other node being primary and forwarded properly

In [ ]:
# For switchover
patronictl -c "patroni.yaml" swithcover --force

### Check logs

- Logs on pg machines
- Logs on HAProxy

### Monitoring

Checkout the REST API of patroni on some node to get information about the cluster

In [ ]:
curl -s http://localhost:8008/patroni | jq .
curl -s http://localhost:8008/cluster | jq .
curl -s http://localhost:8008/history | jq .